<a href="https://colab.research.google.com/github/DarioneNazionale/KickLearning/blob/main/working_on_data/merging_historical_information.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Merging the historical stada in the dataset

### Imports

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
historical_data_path = "./drive/MyDrive/Project/Data"
data_path = "./drive/MyDrive/Project/Data/datasets"
destination_path = "./drive/MyDrive/Project/Data/complete"

In [ ]:
import os
from os.path import join
import pandas as pd
import time

### Fetching the historical data

In [ ]:
historical_df = pd.read_csv(join(historical_data_path, "complete_historical_info.csv"))

### historical Info Function
Function to be then applied on all the dataset

In [ ]:
def get_history(entry, historical_df):
    reduced_historical_df = historical_df[(historical_df["creator_id"] == entry["creator_id"])&(historical_df["id"] != entry["id"])&(historical_df["time"] < entry["launched_at"])]
    reduced_historical_df = reduced_historical_df.loc[reduced_historical_df.groupby('id')['time'].idxmax()]
    reduced_historical_df.groupby('creator_id').apply(list)

    return reduced_historical_df.drop(columns=["id", "creator_id"])

## Applying on all the dataset
Merging the historical data with the actual one file by file

In [ ]:
file_list = sorted(filter(lambda file: file[:4]=="file", os.listdir(data_path)), reverse=True)

start = time.perf_counter()
for i, file_name in enumerate(file_list):

    df = pd.read_csv(os.path.join(data_path, file_name), usecols=["id", "creator_id", "year", "launched_at"])

    df_history = df.apply(lambda entry: get_history(entry, historical_df), axis=1)

    df = pd.concat((df, df_history), axis=1)

    df.to_csv(join(destination_path, file_name), index=False)

    print(f"Time elapsed: {time.perf_counter()-start}; remaning time: {(time.perf_counter()-start)/i*(len(file_list)-i)}")
    print(f"Done {round(i/len(file_list)*100)}% untill now, in total {i} files", end="\n\n")

print(f"Time elapsed in total: {time.perf_counter()-start}")

Time elapsed in total: 1046.721957456
